<center>
![](https://cs9.pikabu.ru/post_img/2017/04/19/9/1492613680130987529.jpg)

In [ ]:
!pip install chart_studio

In [ ]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white")
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from statsmodels.tools.eval_measures import mse, rmse

import warnings
warnings.filterwarnings('ignore')

# pd.options.display.max_columns = 999
%matplotlib inline

from matplotlib import style
style.use('fivethirtyeight')

import chart_studio
chart_studio.tools.set_credentials_file(username='d.kosarevsky', api_key='TY6xzvZdSXn0JiLotHNX')
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Для виз-ии https://streamlit.io ?

https://github.com/infoculture/data-audit-gov-hackathon/issues/23
# Задача
Собрать данные о факторах, влияющих на ожидаемую продолжительность жизни в России и за рубежом. Визуализировать статистику и корреляцию факторов. Изучить альтернативные источники данных о факторах, влияющих на ожидаемую продолжительность жизни. Визуализировать в сравнение ожидаемую продолжительность жизни в России и за рубежом.

## Короткое описание
Увеличение ожидаемой продолжительности жизни является социально ориентированной целью. По данным ВОЗ соответствующий измеримый статистический показатель «Ожидаемая продолжительность жизни при рождении» (далее - ОПЖ) является важнейшим интегральным демографическим показателем, характеризующим уровень смертности населения. Мониторинг ожидаемой продолжительности жизни, общей смертности среди населения и влияющих факторов на ближайшее десятилетие становится важнейшей задачей. Однако, система мониторинга данных показателей в настоящее время отсутствует. Для эффективного принятия решения требуется система позволяющая визуализировать, анализировать и мониторить соответствующие показатели с учётом полной базы данных.

## Источник информации
◦ Справка для Хакатона Сбербанка России (19-20 октября 2019 года). https://drive.google.com/file/d/1zEP7B83be2V71SyhE7-j1i1ZYC_UmbE7/view?usp=sharing

◦ Всемирная организация здравоохранения. https://www.who.int/gho/mortality_burden_disease/life_tables/situation_trends/en/

http://gamapserver.who.int/mapLibrary/app/searchResults.aspx

◦ Всемирный банк https://data.worldbank.org/indicator/sp.dyn.le00.in

◦ Открытые источники https://fedstat.ru/

## Формат
Интерактивная база данных, визуализация, инфографика.

# Изучение данных ВОЗ об ожидаемой продолжительности жизни (ОПЖ)

In [ ]:
woh_data = pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
woh_data.head()

# General Information of the Data

Country : Country

Year : Year

Status : Developed or Developing status

Life expectancy : Life Expectancy in age

Adult Mortality : Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)

infant deaths : Number of Infant Deaths per 1000 population

Alcohol : Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)

Percentage : Expenditure on health as a percentage of Gross Domestic Product per capita(%)

Hepatitis B : Hepatitis B (HepB) immunization coverage among 1-year-olds (%)

Measles : Measles - number of reported cases per 1000 population

BMI : Average Body Mass Index of entire population

Under-five deaths : Number of under-five deaths per 1000 population

Polio : Polio (Pol3) immunization coverage among 1-year-olds (%)

Total expenditure : General government expenditure on health as a percentage of total government expenditure (%)

Diphtheria : Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)

HIV/AIDS : Deaths per 1 000 live births HIV/AIDS (0-4 years)

GDP : Gross Domestic Product per capita (in USD)

Population : Population of the country

Thinness 1-19 years : Prevalence of thinness among children and adolescents for Age 10 to 19 (% )

Income composition of resources : Human Development Index in terms of income composition of resources (index ranging from 0 to 1)

Thinness 5-9 years : Prevalence of thinness among children for Age 5 to 9(%)

Schooling : Number of years of Schooling(years)

In [ ]:
woh_data.columns

Переименуем имена столбцов для будущей работы

In [ ]:
woh_data.columns = ['Страна', 'Год', 'Статус', 'ОПЖ', 'Смертность(взрослые)', 
       'Смертность(младенцы)', 'Алкоголь(потребление)', 'Расходы на здравоохранение (на душу)', 'Гепатит B',
       'Корь', 'ИМТ', 'Смертность (до пяти лет)', 'Полиомиелит', 'Расходы на здравоохранение (общие)',
       'Дифтерия', 'ВИЧ/СПИД', 'ВВП','Население', 'Худоба (1-19 лет)', 'Худоба (5-9 лет)',
       'Доходы(индекс)', 'Образование(лет)']

In [ ]:
woh_data.head()

Загружаем данные всемирного банка

In [ ]:
wbank_data = pd.read_csv('/kaggle/input/world-bank-data-1960-to-2016/life_expectancy.csv')
wbank_data = wbank_data.rename(columns={'Country Name': 'Страна'}).drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)

In [ ]:
wbank_data.sample(5)

# Анализ данных по РФ

### Данные всемирного банка

In [ ]:
# rosstat_data = pd.read_excel('/kaggle/input/rosstat-all/rosstat_all.xls')
# # rosstat_data = rosstat_data.rename(columns={'Ожидаемая продолжительность жизни при рождении (2018, год, значение показателя за год)': 'Регион РФ'})
# rosstat_data.sample(3)

In [ ]:
wbank_data_rus = wbank_data[wbank_data['Страна'] == 'Russian Federation']
wbank_data_rus = wbank_data_rus.set_index('Страна').T.reset_index()
wbank_data_rus.columns=['Год', 'ОПЖ']

In [ ]:
wbank_data_chn = wbank_data[wbank_data['Страна'] == 'China']
wbank_data_chn = wbank_data_chn.set_index('Страна').T.reset_index()
wbank_data_chn.columns=['Год', 'ОПЖ']

In [ ]:
wbank_data_rus_chn = pd.merge(wbank_data_rus, wbank_data_chn, how='left', left_on='Год', right_on='Год', suffixes=('_rus', '_chn'))
wbank_data_rus_chn.head(3)

In [ ]:
china = go.Scatter(
    x=wbank_data_rus_chn['Год'].tolist(),
    y=wbank_data_rus_chn['ОПЖ_chn'].tolist(), name='Китай',
)
russia = go.Scatter(
    x=wbank_data_rus_chn['Год'].tolist(),
    y=wbank_data_rus_chn['ОПЖ_rus'].tolist(), name='Россия',
)

data = [china, russia]

py.iplot(data)

In [ ]:
wbank_data_usa = wbank_data[wbank_data['Страна'] == 'United States']
wbank_data_usa = wbank_data_usa.set_index('Страна').T.reset_index()
wbank_data_usa.columns=['Год', 'ОПЖ']

In [ ]:
wbank_data_rus_usa = pd.merge(wbank_data_rus, wbank_data_usa, how='left', left_on='Год', right_on='Год', suffixes=('_rus', '_usa'))
wbank_data_rus_usa.head(3)

In [ ]:
usa = go.Scatter(
    x=wbank_data_rus_usa['Год'].tolist(),
    y=wbank_data_rus_usa['ОПЖ_usa'].tolist(), name='США',
)
russia = go.Scatter(
    x=wbank_data_rus_usa['Год'].tolist(),
    y=wbank_data_rus_usa['ОПЖ_rus'].tolist(), name='Россия',
)
data = [usa, russia]

py.iplot(data)

In [ ]:
fig = px.line(wbank_data_rus, x='Год', y='ОПЖ')
fig.update_layout(
    title=go.layout.Title(
        text="Ожидаемая продолжительность жизни в России начиная с 1960 года (Данные всемирного банка)",
        xref="paper",
        x=0
    ),
)
fig.show()

## Анализ данных ВОЗ

In [ ]:
woh_data_jpn = woh_data[woh_data['Страна'] == 'Japan']
woh_data_jpn.shape

In [ ]:
woh_data_can = woh_data[woh_data['Страна'] == 'Canada']
woh_data_can.shape

In [ ]:
woh_data_rus = woh_data[woh_data['Страна'] == 'Russian Federation']
woh_data_rus.shape

In [ ]:
years = woh_data_rus['Год'].tolist()

fig = go.Figure()
fig.add_trace(go.Bar(x=years,
                y=woh_data_rus['ОПЖ'].tolist(),
                name='Россия',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=years,
                y=woh_data_jpn['ОПЖ'].tolist(),
                name='Япония',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='ВОЗ, сравнение ожидаемой продолжительности жизни России и Японии',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='ОПЖ',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=.8,
        y=1.1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [ ]:
years = woh_data_rus['Год'].tolist()

fig = go.Figure()
fig.add_trace(go.Bar(x=years,
                y=woh_data_rus['ОПЖ'].tolist(),
                name='Россия',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=years,
                y=woh_data_can['ОПЖ'].tolist(),
                name='Канада',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='ВОЗ, сравнение ожидаемой продолжительности жизни России и Канады',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='ОПЖ',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [ ]:
woh_data_rus.sample(3)

## Визуализация корреляций

In [ ]:
woh_data_rus.columns

In [ ]:
fig = px.bar(woh_data_rus, x='Год', y='ВИЧ/СПИД',
             hover_data=['ОПЖ', 'ВВП'], color='ОПЖ',
             labels={'ВИЧ/СПИД':'ВИЧ/СПИД России'}, height=400)
fig.update_layout(
    title='Корреляция ВИЧ/СПИД и ВВП на ожидание продолжительности жизни',)
fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Bar(
#     x=woh_data_rus['Год'].tolist(),
#     y=woh_data_rus['ОПЖ'].tolist(),
#     name='Смертность(младенцы)',
#     marker_color='indianred'
# ))
# fig.add_trace(go.Bar(
#     x=woh_data_rus['Год'].tolist(),
#     y=woh_data_rus['ОПЖ'].tolist(),
#     name='Смертность (взрослые)',
#     marker_color='lightsalmon'
# ))

# # Here we modify the tickangle of the xaxis, resulting in rotated labels.
# fig.update_layout(barmode='group', xaxis_tickangle=-45)
# fig.show()

In [ ]:
# plt.figure(figsize=(5,5))
# sns.distplot(woh_data_rus[woh_data_rus['ОПЖ'] != np.nan]['Смертность(младенцы)'].dropna(), axlabel='Корреляция ОПЖ и младенческой смертности, Россия')
# plt.show()

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(woh_data_rus.corr(), annot=True, cmap='RdBu_r')
plt.title('Матрица корреляции факторов влияющих на ожидаемую продолжительность жизни в России');

Как мы видим выше, **Расходы на здравоохранение (общие)**, **Доходы(индекс)**, **ВВП** и **Образование** имеют высокую корреляцию, в то время как **Младенческая смертность**, **Смертность до пяти лет**, **ВИЧ/СПИД** и **Худоба(в возрасте от 5 до 9 лет)** имеет высокую отрицательную корреляцию между продолжительностью жизни.

**ИМТ** и **Худоба(в возрасте до 19 лет)** имеют среднюю корреляцию между ожидаемой продолжительностью жизни.

**Смертность(взрослые)**, **Корь**, **Потребление алкоголя**, **Расходы на здравоохранение (на душу населения)**, **Гепатит B**, **Полиомиелит**, **Дифтерия** и **Население** имеют низкую корреляцию между ожидаемой продолжительностью жизни.

# Корреляции между заболеваниями

In [ ]:
woh_data_rus.head(2)

In [ ]:
woh_data_rus_ill = woh_data_rus[['Гепатит B','Корь', 'Полиомиелит','Дифтерия','ВИЧ/СПИД', 'Худоба (1-19 лет)',
                                      'Худоба (5-9 лет)', 'ИМТ', 'ОПЖ']]

plt.figure(figsize=(8,8))
sns.heatmap(woh_data_rus_ill.corr(), annot=True, cmap='RdBu_r')
plt.title('Матрица корреляции между заболеваниями и ожидаемой продолжительностью жизни в России');

Ожидаемая продолжительность жизни имеет высокую отрицательную корреляцию с **ВИЧ/СПИДом** и **Худобой(в возрасте от 5 до 9 лет)** если мы смотрим корреляции только на основе заболеваний

Также мы видим среднюю корреляцию с **Худобой(в возрасте до 19 лет)**, **Гепатитом B** и **ИМТ**.

Ожидаемая продолжительность жизни имеет низкую корреляцию с **Корью**, **Полиомиелитом** и **Дифтерией**.

In [ ]:
fig = px.line(woh_data_rus, x='Год', y='ОПЖ')
fig.update_layout(
    title=go.layout.Title(
        text="Ожидаемая продолжительность жизни в России начиная с 2000 года (Данные ВОЗ)",
        xref="paper",
        x=0
    ),
)
fig.show()

# Анализ ситуации в мире и регионах

In [ ]:
fig = px.scatter_matrix(woh_data, dimensions=['Худоба (1-19 лет)', 'Худоба (5-9 лет)', 'Доходы(индекс)', 'ВИЧ/СПИД', 'Смертность(младенцы)'], color='ОПЖ')
fig.update_layout(
    title='Корреляция Худобы, ВИЧ/СПИД, Доходов и Смертности среди младенцев на ожидание продолжительности жизни',)
fig.show()

In [ ]:
fig = px.parallel_categories(woh_data, color="Год", color_continuous_scale=px.colors.sequential.Inferno)
fig.update_layout(
    title='Зависимость между годом и статусом страны',)
fig.show()

In [ ]:
x0 = woh_data['Худоба (5-9 лет)'].tolist()
# Add 1 to shift the mean of the Gaussian distribution
x1 = woh_data['Доходы(индекс)'].tolist()

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name='Худоба'))
fig.add_trace(go.Histogram(x=x1, name='Доходы'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
woh_data.columns

In [ ]:
fig = px.box(woh_data, x="Год", y="ОПЖ",
             notched=True, # used notched shape
             title="Обзор ожидания продолжительности жизни в мире",
#              hover_data=["day"] # add day column to hover data
            )
fig.show()

Мы видим на графике, как общее значение ожидаемой продолжительности жизни снижается после 2010 года до 2014 года.

Для того, чтобы разобраться в причинах поведения цифр нужно изучить данные на более глубоком уровне.
Давайте посмотрим на значения продолжительности жизни в годах по регионам

Загрузим дополнительные данные по регионам (материки) и субрегионам

In [ ]:
regions = pd.read_csv('/kaggle/input/countryinfo/country-continent.csv')
regions = regions[['name', 'region', 'sub-region']]
regions.columns = ['Страна', 'Регион', 'Субрегион']

In [ ]:
# # woh_data['Страна'].nunique()
# woh_cntrs = woh_data['Страна'].unique().tolist()
# woh_cntrs.sort()
# woh_cntrs[:5]

# regions_cntrs = regions['Страна'].unique().tolist()
# regions_cntrs.sort()
# regions_cntrs[:5]

# result_cntrs1 = list(set(woh_cntrs) - set(regions_cntrs))
# result_cntrs1

In [ ]:
regions['Страна'] = regions['Страна'].replace('Congo (Democratic Republic of the)', 'Democratic Republic of the Congo')
regions['Страна'] = regions['Страна'].replace('Moldova (Republic of)', 'Republic of Moldova')
regions['Страна'] = regions['Страна'].replace('Korea (Republic of)', 'Republic of Korea')
regions['Страна'] = regions['Страна'].replace('Macedonia (the former Yugoslav Republic of)', 'The former Yugoslav republic of Macedonia')
regions['Страна'] = regions['Страна'].replace("Korea (Democratic People's Republic of)", "Democratic People's Republic of Korea")
regions['Страна'] = regions['Страна'].replace('Tanzania, United Republic of', 'United Republic of Tanzania')

In [ ]:
# woh_cntrs = woh_data['Страна'].unique().tolist()
# woh_cntrs.sort()
# regions_cntrs = regions['Страна'].unique().tolist()
# regions_cntrs.sort()

# result_cntrs1 = list(set(woh_cntrs) - set(regions_cntrs))
# result_cntrs1

In [ ]:
woh_data_regions = pd.merge(woh_data, regions, how='left', left_on='Страна', right_on='Страна')
woh_data_regions.sample(3)

In [ ]:
fig = px.box(woh_data_regions.loc[woh_data_regions['Год'] > 2009], x="Год", y="ОПЖ", color="Регион",
             notched=True, # used notched shape
             title="Значения продолжительности жизни по континентальным регионам с 2010 года",
            )
fig.show()

Мы видим, что у жителей регионов Африки и Азии стабильная продолжительность жизни в целом, в то время как в регионах Океании и Европы наблюдается тенденция к снижению в период с 2010 по 2014 годы.

## Анализ ожидаемой продолжительности жизни по группам стран со значениями ВВП на основе и  регионов c акцентом на младенческую смертность

In [ ]:
woh_data_regions.columns

In [ ]:
fig = px.scatter(woh_data_regions, x="ВВП", y="ОПЖ",
	         size="Смертность(младенцы)", color="Регион",
                 hover_name="Страна", log_x=True, size_max=60)
fig.show()

## Анализ ожидаемой продолжительности жизни по группам стран со значениями ВВП на основе и  регионов c акцентом на заболевания СПИД/ВИЧ

In [ ]:
fig = px.scatter(woh_data_regions, x="ВВП", y="ОПЖ",
	         size="ВИЧ/СПИД", color="Регион",
                 hover_name="Страна", log_x=True, size_max=60)
fig.show()

## Анализ ожидаемой продолжительности жизни по группам стран со значениями ВВП на основе и  регионов c акцентом на расходы на здравоохранение (на душу)

In [ ]:
fig = px.scatter(woh_data_regions, x="ВВП", y="ОПЖ",
	         size="Расходы на здравоохранение (на душу)", color="Регион",
                 hover_name="Страна", log_x=True, size_max=60)
fig.show()

# Добавим в наши данные новые - мировой индекс счастья

In [ ]:
happines_2015 = pd.read_csv('/kaggle/input/world-happiness/2015.csv')
happines_2015.sample(3)

Данные за 2015 год, тем не менее можно попробовать найти интересные корреляциии. 
Вместе с индексом счастья в датасете есть и другие интересные признаки, а также есть и повторяющиеся с нашим основным датасетом, их уберём.

In [ ]:
happines_2015.columns=(['Страна', 'Регион', 'Индекс счастья', 'Оценка счастья', 'Стандартная ошибка', 'ВВП на душу населения', 
'Семья', 'ОПЖ', 'Свобода', 'Доверие правительству', 'Щедрость', 'Остаточная дистопия'])
happines_2015 = happines_2015.drop(columns=['Регион', 'ВВП на душу населения', 'ОПЖ'])

In [ ]:
happines_2015.sample(3)

In [ ]:
# happ_cntrs = happines_2015['Страна'].unique().tolist()
# happ_cntrs.sort()

# regions_cntrs = woh_data_regions['Страна'].unique().tolist()
# regions_cntrs.sort()

In [ ]:
happines_2015['Страна'] = happines_2015['Страна'].replace('Bolivia', 'Bolivia (Plurinational State of)')
happines_2015['Страна'] = happines_2015['Страна'].replace('Somaliland region', 'Somalia')
happines_2015['Страна'] = happines_2015['Страна'].replace('Moldova', 'Republic of Moldova')
happines_2015['Страна'] = happines_2015['Страна'].replace('Congo (Kinshasa)', 'Democratic Republic of the Congo')
happines_2015['Страна'] = happines_2015['Страна'].replace('United States', 'United States of America')
happines_2015['Страна'] = happines_2015['Страна'].replace('Macedonia', 'The former Yugoslav republic of Macedonia')
happines_2015['Страна'] = happines_2015['Страна'].replace('United Kingdom', 'United Kingdom of Great Britain and Northern Ireland')
happines_2015['Страна'] = happines_2015['Страна'].replace('Venezuela', 'Venezuela (Bolivarian Republic of)')
happines_2015['Страна'] = happines_2015['Страна'].replace('Iran', 'Iran (Islamic Republic of)')
happines_2015['Страна'] = happines_2015['Страна'].replace('Congo (Brazzaville)', 'Congo')
happines_2015['Страна'] = happines_2015['Страна'].replace('Syria', 'Syrian Arab Republic')
happines_2015['Страна'] = happines_2015['Страна'].replace('Russia', 'Russian Federation')
happines_2015['Страна'] = happines_2015['Страна'].replace('Tanzania', 'United Republic of Tanzania')

In [ ]:
# result_cntrs1 = list(set(happ_cntrs) - set(regions_cntrs))
# result_cntrs1

In [ ]:
# result_cntrs2 = list(set(regions_cntrs) - set(happ_cntrs))
# result_cntrs2

In [ ]:
woh_data_regions_happ = pd.merge(woh_data_regions, happines_2015, how='left', left_on='Страна', right_on='Страна')
woh_data_regions_happ.sample(3)

Объединив две таблицы нарисуем ещё одну матрицу корреляций, чтобы увидеть изменения.

In [ ]:
woh_data_regions_happ.columns

In [ ]:
woh_data_regions_happ_corr = woh_data_regions_happ[['Страна', 'Регион', 'Оценка счастья', 'Стандартная ошибка', 'ВВП', 
'Семья', 'ОПЖ', 'Свобода', 'Доверие правительству', 'Щедрость', 'Остаточная дистопия']]

plt.figure(figsize=(8,8))
sns.heatmap(woh_data_regions_happ_corr.corr(), annot=True, cmap='RdBu_r')
plt.title('Матрица корреляции с новыми признаками в разрезе всех стран');

Оценка счастья положительно коррелирует с ОПЖ. Наличие семьи показывает средние показатели.
Отрицательных значений с новыми признаками не наблюдаем.

Стоит отметить, что наличие семьи положительно влияет на оценку счастья.

Другие признаки имеют низкую (или ниже среднего) корреляцию с ожидаемой продолжительностью жизни.

In [ ]:
# ['Asia', 'Europe', 'Africa', 'Americas', 'Oceania']

## И немного подытожим по регионам

ОПЖ Океания

Новая Зеландия(82), Автралия(81) и Самоа(74)

In [ ]:
fig = px.line(woh_data_regions_happ[woh_data_regions_happ['Регион'] == 'Oceania'], x='Год', y='ОПЖ', color='Страна')
fig.show()

ОПЖ Америка 

Впереди Чили(85), Канада(82) и Коста Рика(79)

In [ ]:
fig = px.line(woh_data_regions_happ[woh_data_regions_happ['Регион'] == 'Americas'], x='Год', y='ОПЖ', color='Страна')
fig.show()

ОПЖ Африка

Египет(79), Алжир(75) и Тунис(75)

In [ ]:
fig = px.line(woh_data_regions_happ[woh_data_regions_happ['Регион'] == 'Africa'], x='Год', y='ОПЖ', color='Страна')
fig.show()

Азия

Япония(83), Сингапур(83) и Израиль(82)

In [ ]:
fig = px.line(woh_data_regions_happ[woh_data_regions_happ['Регион'] == 'Asia'], x='Год', y='ОПЖ', color='Страна')
fig.show()

Европа

Словения (88 лет), Дания(86) и Швейцария(83)

In [ ]:
fig = px.line(woh_data_regions_happ[woh_data_regions_happ['Регион'] == 'Europe'], x='Год', y='ОПЖ', color='Страна')
fig.show()

Наша страна не так давно была аутсайдером по показателю ожидаемой продолжительности жизни среди европейских стран, где данный показатель составляет порядка 70 лет. Однако, в поледние годы мы делаем значительные успехи в этой области благодаря различным факторам, таким, как улучшение здравоохранения, снижение роста имунных заболеваний, увеличения ВВП и других.
В то же время, по-прежнему остро стоит проблема с младенческой смертью (уверен, не зря на государственном уровне решается задача по строительству высокотехнологичных перинатальных центров) и проблема низкого индекса массы тела (худобы) среди детей в следствие низкого индекса доходов (вероятно, ведущую роль в этом играет проблема бедности)

Помимо анализа данных, который, несомненно, может принести пользу, намного более перспективным представляется использование созданной на основе собранных данных модели. Бесспорно, этих данных не  достаточно для полноценной и объективной картины, необходимо добавить также важные и влияющие напрямую на показатель ОПЖ данные, касающиеся экологии, преступности, вооруженных конфликтов, охране и безопасности труда с уровнем производственных рисков, связанных с травматизацией и утратой трудоспособности, и других параметров, оказывающих непосредственное влияние на ОПЖ. Тем не менее, создав качественную модель и применив к ней методы машинного обучения (для начала вполне подойдёт линейная регрессия), мы сможем делать предсказания по уровню роста ОПЖ и качественно влиять на те или иные факторы для того, чтобы увеличить ОПЖ и, как следствие, уровень благополучия населения.

# Будьте здоровы!

# И помните...

<center>
![](https://onefile.space/sl-1/ec4be147f531df6b539e/2019-10-21_10-33-27.png)